In [ ]:
# HOME
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
# use seaborn plotting defaults
import seaborn as sns; sns.set()

# get the code
import sys
sys.path.append('../scripts')
from codeCNV.plot import plot_snp, plot_2d, plot_3d
from codeCNV.rollingCNV import apply_rolling_SNP, llh

home = '/Users/mahtin'
home = '/Users/martinscience'
# standard paths
testdata = os.path.join(home,"Dropbox/Icke/Work/somVar/testdata")
static_path = os.path.join(home, "Dropbox/Icke/Work/static")
cluster_path = os.path.join(home, "mount")
tooldata = os.path.join(home, "Dropbox/Icke/Work/somVar/tooldata")

# load the config
# edit config directly in yaml file
import yaml
config_file = '../config/config_devel.yaml'
def get_config(config_file, param):
        with open(config_file) as file:
        # The FullLoader parameter handles the conversion from YAML
        # scalar values to Python the dictionary format
            config = yaml.load(file, Loader=yaml.FullLoader)['CNV'][param]
        return config
config = get_config(config_file, 'combine')


cnvdata = os.path.join(tooldata, "myCNVdata")
output_path = os.path.join(cnvdata, "output")

# the path to the input data
cnv_path = os.path.join(cnvdata, "cnv")
cnvPON_path = cnv_path = os.path.join(cnvdata, "chromCov")
cnv_path = os.path.join(cluster_path, "scratch/develop/PONcoverage/cnv")
cnvPON_path = os.path.join(cluster_path, "scratch/develop/PONcoverage/chromCov")

## get the SNP data with FullExonPos

In [ ]:
sample = "02_A"
snp_df = pd.read_csv(os.path.join(output_path, f'tmp/{sample}.snpcov.gz'), sep='\t', compression='gzip')

In [ ]:
log2 = dict(
        title='log2ratio',
        plot_type='scatter',   # ['line', 'scatter']
        data='log2ratio',
        plot_args=dict(
            linewidth=0.3,
            color='black',
            s=0.2,
            alpha=.7
        )
    )

log2mean = dict(
        title='rollinglog2ratio',
        plot_type='line',   # ['line', 'scatter']
        data='log2ratiomean',
        plot_args=dict(
            linewidth=1,
            color='yellow',
            alpha=.7
        )
    )

vaf = dict(
        title='VAF',
        plot_type='scatter',  # ['line', 'scatter']
        data='VAF',
        plot_args=dict(
            s=2,
            color='black',
            alpha=1
        )
    )

fig_params = dict(
    figsize=(24,8),
    colormap='coolwarm_r',
    color_chroms=True,
    ylim=(0,1),
    cov_offset=.1,  # how much log2ratio=0 is shifted above SNP-data
    cov_height=.5,
    label_size=13
)
chroms = ['chr5', 'chr7','chr8', 'chr11', 'chr17']
r1 = 'chr17:3Mb-9Mb'

fig, ax, df, chrom_df = plot_snp(snp_df, snp_plots=[vaf], cov_plots=[log2, log2mean], chroms='all', region='', **fig_params)

### strategy for finding cluster
+ look at one chrom
+ get the mean from the middle range (0.4 < VAF < 0.6)
+ compute loglikelihood for samples
+ compute rolling sum 

In [ ]:
import numpy.random as rnd


# chr17 only
df = snp_df.query('Chr == "chr17"')

sigma = 0.02
# get the mean of the center band
mean = df.query('0.4 < VAF < 0.6')['VAF'].mean()
mean

df2 = df.query('@mean - 0.1 < VAF < @mean + 0.1')
def llh(data, mean, sigma):
    s = np.sqrt(2 * np.pi) * sigma
    return np.exp((data - mean)**2 / (-2*(sigma**2))) / s
r = np.linspace(0,1,1000000)
plt.scatter(r, llh(r, mean, sigma), s=.01, alpha=0.1);
plt.scatter(r, llh(r, 1, 0.02), s=.01, alpha=0.1);
plt.scatter(df.iloc[:1000,:]['VAF'], rnd.random(1000)*10, s=1)
df2.loc[:, 'llh'] = llh(df2['VAF'], mean, sigma)


df2.loc[:, 'LLH'] = df2['llh'].rolling(50).sum()
df2.loc[:, 'LLH'] = df2['LLH'] / df2['LLH'].max()
df2.loc[:, 'absVAF'] = np.abs(df2['VAF'] - mean) * 2

In [ ]:
center_vaf = snp_df.query('0.4 < VAF < 0.6')['VAF']
print(f"std={center_vaf.std()}")
print(f"mean={center_vaf.mean()}")

In [ ]:
def compute_snp_llh(df, config):
    '''
    computes the local log-likelihood of belonging to the center gaussian
    '''
    
    # get config params
    params = config['snp']['LLH']
    
    # get sigma and mean for llh computation
    minVAF, maxVAF = params['center_range']
    # get the sigma and mean of the center band VAF (extracted as pd.Series center_vafs)
    center_vafs = df.query('@minVAF < VAF < @maxVAF')['VAF']
    # get width of gaussian from std * sigma_factor
    sigma = center_vafs.std() * params['sigma_factor']
    mean = center_vafs.mean()
    print(f"Computing log-likelihood of VAF belonging to center gaussian [mean:{round(mean, 3)}, sigma:{round(sigma,3)}]")
    df.loc[:, 'snpLLH'] = llh(df['VAF'], mean, sigma)
    print(f"Computing log-likelihood of VAF belonging to purity100  [mean:1, sigma:{round(sigma,3)}]")
    # these are called hsnp
    df.loc[:, 'hsnpLLH'] = llh(df['VAF'], 1, sigma)
    return df, mean

In [ ]:
df, mean = compute_snp_llh(snp_df, config)
df

In [ ]:
absvaf = dict(
        title='absVAF',
        plot_type='scatter',  # ['line', 'scatter']
        data='absVAF',
        plot_args=dict(
            s=2,
            color='blue',
            alpha=.7
        )
    )


LLH = dict(
        title='log2ratio',
        plot_type='line',   # ['line', 'scatter']
        data='LLH',
        plot_args=dict(
            linewidth=1,
            color='yellow',
            alpha=.7
        )
    )

_ = plot_snp(df2, snp_plots=[absvaf, LLH], chroms='all', region='', **fig_params)

In [ ]:
from sklearn.mixture import GaussianMixture as GMM
from sklearn.metrics import silhouette_score as ssc

xcol = 'VAF'
ycol = "log2ratiomean"
df = snp_df.query('Chr == "chr17" and VAF > 0.02').iloc[:500,:]
select_cols = ['VAF']
X = df[select_cols]

model = GMM(n_components=4, covariance_type='diag', n_init=20).fit(X)
labels = model.predict(X)
fig, ax = plt.subplots()

_ = ax.scatter(df[xcol], df[ycol], c=labels, s=1, cmap='rainbow')
_ = ax.set_xlabel(xcol, fontsize=10)
_ = ax.set_ylabel(ycol, fontsize=10)
ssc(X, labels, metric='euclidean')

In [ ]:
model.means_

In [ ]:
model.covariances_